In [1]:
import os
os.chdir("../")
%pwd

'/home/anonymous/PycharmProjects/MLOps_heart_disease'

In [2]:
from dataclasses import dataclass
from pathlib import Path

In [3]:
@dataclass(frozen=True)
class ModelTrainerConfig:
    root_dir: Path
    train_data_path: Path
    test_data_path: Path
    model_name: str
    alpha: float
    l1_ratio: float
    target_column: str

In [4]:
from mlhrtds.constants import *
from mlhrtds.utils.common import read_yaml, create_directories

In [17]:
class ConfigurationManager:
    def __init__(
        self,
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH,
        schema_filepath = SCHEMA_FILE_PATH):

        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        self.schema = read_yaml(schema_filepath)

        create_directories([self.config.artifacts_root])


    def get_model_trainer_config(self) -> ModelTrainerConfig:
        config = self.config.model_trainer
        params = self.params.CatBoostRegressor
        schema =  self.schema.TARGET_COLUMN

        create_directories([config.root_dir])

        model_trainer_config = ModelTrainerConfig(
            root_dir=config.root_dir,
            train_data_path = config.train_data_path,
            test_data_path = config.test_data_path,
            model_name = config.model_name,
            alpha = params.alpha,
            l1_ratio = params.l1_ratio,
            target_column = schema.name
            
        )

        return model_trainer_config

In [18]:
import pandas as pd
import os
from mlhrtds import logger
import joblib
from catboost import CatBoostRegressor

In [13]:
train_data = pd.read_csv("/home/anonymous/PycharmProjects/MLOps_heart_disease/artifacts/data_transformation/X_train.csv")
test_data = pd.read_csv("/home/anonymous/PycharmProjects/MLOps_heart_disease/artifacts/data_transformation/y_train.csv")

In [23]:
class ModelTrainer:
    def __init__(self, config: ModelTrainerConfig):
        self.config = config

    
    def train(self):
        train_data = pd.read_csv(self.config.train_data_path)
        test_data = pd.read_csv(self.config.test_data_path)


        train_x = train_data
        # test_x = test_data.drop([self.config.target_column], axis=1)
        train_y = test_data
        print(train_y)
        # test_y = test_data[[self.config.target_column]]


        lr = CatBoostRegressor(verbose=False)
        lr.fit(train_x, train_y)

        joblib.dump(lr, os.path.join(self.config.root_dir, self.config.model_name))


In [24]:
try:
    config = ConfigurationManager()
    model_trainer_config = config.get_model_trainer_config()
    model_trainer = ModelTrainer(config=model_trainer_config)
    model_trainer.train()
    
except Exception as e:
    raise e

[2023-09-20 21:24:35,455: INFO: common: yaml file: config/config.yaml loaded successfully]
[2023-09-20 21:24:35,458: INFO: common: yaml file: params.yaml loaded successfully]
[2023-09-20 21:24:35,461: INFO: common: yaml file: schema.yaml loaded successfully]
[2023-09-20 21:24:35,463: INFO: common: created directory at: artifacts]
[2023-09-20 21:24:35,464: INFO: common: created directory at: artifacts/model_trainer]
          charges
0      2352.96845
1      4340.44090
2      9391.34600
3     42211.13820
4      8823.27900
...           ...
998    4561.18850
999    8582.30230
1000  11931.12525
1001  46113.51100
1002  10214.63600

[1003 rows x 1 columns]
